<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

In [2]:
#leeftijdsgroepen = SimpleNamespace()
@run
def cell():
  global rivm
  rivm = RIVM.csv('COVID-19_casus_landelijk')
  display(rivm.head())

rivm/COVID-19_casus_landelijk-2022-01-06@14-15.csv exists


loading rivm/COVID-19_casus_landelijk-2022-01-06@14-15.csv.gz


,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Hospital_admission,Deceased,Week_of_death,Municipal_health_service
0,2022-01-06 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,No,NaN,GGD Amsterdam
1,2022-01-06 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2022-01-06 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,No,NaN,GGD Hollands-Midden
3,2022-01-06 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,No,NaN,GGD Hollands-Noorden
4,2022-01-06 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,Unknown,No,NaN,GGD Gelderland-Zuid


In [3]:
@run
def cell():
  global bevolking

  # December 2021 als backup:
  aantallenpercohort = {'0-9': 1757736,
    '10-19': 1981885,
    '20-29': 2263186,
    '30-39': 2213705,
    '40-49': 2137833,
    '50-59': 2550358,
    '60-69': 2171183,
    '70-79': 1649291,
    '80-89': 722027,
    '90+': 135289}

  global bevolking

  bevolking = None
  try:
    bevolking = CBS.bevolking(leeftijdsgroepen=True)
    bevolking.to_csv("leeftijdsgroepen_cbs.csv")
  except Exception as e:
    print(e)
    # gebruik laatste csv
    try:
      bevolking = pd.read_csv("leeftijdsgroepen_cbs.csv")
      bevolking.set_index('Range', inplace=True)
    except Exception as ee:
      print(ee)
      bevolking = None

  if bevolking is None:
    bevolking = aantallenpercohort

In [4]:
# Bereken de stand van zaken van besmettingen / hospitalisaties / overlijden, per cohort in absolute aantallen en aantallen per 100k, met een kleur indicator voor de aantallen.
# vervang <50 en Unknown door Onbekend
@run
def cell():
  rivm['Cohort'] = rivm['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
  # aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
  assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in rivm['Cohort'].unique() if cohort[0].isdigit()]
  assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
  rivm['Gemiddelde leeftijd'] = rivm['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

  # verwijder tijd
  rivm['Date_file_date'] = pd.to_datetime(rivm['Date_file'].replace(r' .*', '', regex=True))

  rivm['Date_statistics_date'] = pd.to_datetime(rivm['Date_statistics'])

  # weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
  rivm['Weken terug'] = np.floor((rivm['Date_file_date'] - rivm['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(int)

  # voeg key, gem leeftijd, kleurnummer en totaal toe
  Date_file = rivm['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
  # check type bevolking
  if type(bevolking) == dict:
    cohorten = list(bevolking.keys())
  else:
    cohorten = list(bevolking.index)
  cohorten += ['Onbekend']

  def summarize(df, category, prefix):
    # aangezien we hier de dataframe in-place wijzigen (bijv door toevoegen kolommen)
    # en we het 'rivm' frame later nog clean nodig hebben
    df = df.copy(deep=True)

    df = (df
          .groupby(['Weken terug', 'Cohort'])['count']
          .sum()
          .unstack(fill_value=np.nan)
          .reset_index()
          .rename_axis(None, axis=1)
        ).merge(df
          # we voegen hier gemiddelde leeftijd toe, want die willen we op een ander
          # niveau aggregeren voor 'df' overschreven word
          .groupby(['Weken terug'])['Gemiddelde leeftijd']
          .mean()
          .to_frame(), on='Weken terug'
        )

    # altijd 52 rijen
    df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df, how='left', on='Weken terug')

    # toevoegen missende cohorten
    for col in cohorten:
      if not col in df:
        df[col] = np.nan

    # sommeer per rij (axis=1) over de cohorten om een totaal te krijgen
    df['Totaal'] = df[cohorten].sum(axis=1)

    # voeg periode en datum toe
    # periode afgeleid van weken-terug (= de index voor deze dataframe)
    df['Datum'] = pd.to_datetime(Date_file)
    df['Periode'] = (df
      .index.to_series()
      .apply(
        lambda x: (
          (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m')
          + '-'
          + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
        )
      )
    )

    # voeg 'Key' en 'Type' kolom toe. Variabele 'type' kan niet, is een language primitive.
    df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
    df['Type'] = category

    # voeg de kleur kolommen toe
    for col in cohorten:
      df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

    # herschikken van de kolommen
    colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
    return df[colorder + [col for col in df if col not in colorder]]

  if type(bevolking) == dict:
    factor = {}
    for c in bevolking: factor[c] = 100_000 / bevolking[c]
  else:
    factor = bevolking.to_dict()['per 100k']

  global tabel
  tabel = pd.concat(
    # flatten the result list zodat pd.concat ze onder elkaar kan plakken
    functools.reduce(lambda a, b: a + b, [
      [summarize(df.assign(count=1), label, prefix), summarize(df.assign(count=df['Cohort'].apply(lambda x: factor.get(x, np.nan))), label + ' per 100.000', prefix + '100k')]
      for df, label, prefix in [
        (rivm, 'Positief getest', 'p'), # volledige count per cohort
        (rivm[rivm.Hospital_admission == 'Yes'], 'Ziekenhuisopname', 'h'), # count van cohort voor Hospital_admission == 'Yes'
        (rivm[rivm.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
      ]
    ])
  )

  if type(bevolking) == dict:
    totale_bevolking = sum(bevolking.values())
    print(totale_bevolking)
  else:
    totale_bevolking_per_cohort = bevolking.to_dict()['BevolkingOpDeEersteVanDeMaand']
    totale_bevolking = sum( totale_bevolking_per_cohort.values())

  keys = [ key for key in tabel["Key"]]
  totals = [ x for x in tabel["Totaal"]]
  for k in range(0, len(keys)):
    key = keys[k]
    if '100k' in key:
      abskey = key[0] + key[5:]
      kk = keys.index(abskey)
      if isinstance(totals[k], float):
        correctedtotal = totals[kk] * (100_000 / totale_bevolking)
        # print([k, totals[k], totals[kk], correctedtotal])
        totals[k] = correctedtotal

  tabel["Totaal"] = totals


  # rood -> groen
  cdict = {
    'red':   ((0.0, 0.0, 0.0),   # no red at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.8, 0.8)),  # set to 0.8 so its not too bright at 1
    'green': ((0.0, 0.8, 0.8),   # set to 0.8 so its not too bright at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0)),  # no green at 1
    'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0))   # no blue at 1
  }
  cm = colors.LinearSegmentedColormap('GnRd', cdict)
  # geel -> paars
  cm = sns.color_palette('viridis_r', as_cmap=True)
  display(tabel
    .fillna(0)
    .head()
    .round(1)
    .reset_index(drop=True)
    .style.background_gradient(cmap=cm, axis=1, subset=cohorten)
  )

,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Onbekend,c0-9,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2022-01-06 00:00:00,31/12-06/01,33.800000,86357.000000,Positief getest,3620.000000,13443.000000,27571.000000,13887.000000,10420.000000,10697.000000,4328.000000,1735.000000,505.000000,151.000000,0.000000,131,487,1000,503,377,387,156,62,18,5,0
1,p001,1,2022-01-06 00:00:00,24/12-30/12,37.500000,94608.000000,Positief getest,6182.000000,11526.000000,20133.000000,16714.000000,13476.000000,13944.000000,7932.000000,3512.000000,966.000000,223.000000,0.000000,307,572,1000,830,669,692,393,174,47,11,0
2,p002,2,2022-01-06 00:00:00,17/12-23/12,35.700000,85832.000000,Positief getest,10428.000000,11651.000000,14359.000000,14795.000000,12597.000000,10155.000000,6940.000000,3385.000000,1200.000000,321.000000,1.000000,704,787,970,1000,851,686,469,228,81,21,0
3,p003,3,2022-01-06 00:00:00,10/12-16/12,36.300000,97734.000000,Positief getest,13597.000000,14371.000000,12641.000000,15805.000000,14442.000000,11071.000000,8143.000000,4832.000000,2197.000000,631.000000,4.000000,860,909,799,1000,913,700,515,305,139,39,0
4,p004,4,2022-01-06 00:00:00,03/12-09/12,36.300000,126060.000000,Positief getest,18322.000000,19153.000000,14728.000000,20587.000000,18698.000000,13772.000000,10342.000000,6195.000000,3278.000000,983.000000,2.000000,889,930,715,1000,908,668,502,300,159,47,0


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
if knack:
  await knack.publish(tabel.fillna(0).assign(Datum=tabel.Datum.dt.strftime('%Y-%m-%d')), 'Leeftijdsgroep', Cache)

infinities:


Empty DataFrame
Columns: []
Index: []


nan:


Empty DataFrame
Columns: []
Index: []


Key                     object
Weken terug              int64
Datum                   object
Periode                 object
Gemiddelde leeftijd    float64
Totaal                 float64
Type                    object
0-9                    float64
10-19                  float64
20-29                  float64
30-39                  float64
40-49                  float64
50-59                  float64
60-69                  float64
70-79                  float64
80-89                  float64
90+                    float64
Onbekend               float64
c0-9                     int64
c10-19                   int64
c20-29                   int64
c30-39                   int64
c40-49                   int64
c50-59                   int64
c60-69                   int64
c70-79                   int64
c80-89                   int64
c90+                     int64
cOnbekend                int64
dtype: object
updating knack


...

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.56s/it]

100%|██████████| 1/1 [00:04<00:00,  4.57s/it]


rate limit: 7 
LaatsteUpdate API calls: create: 0, read: 1, update: 1, delete: 0, backoff: 0


JSONDecodeError: [Errno Unterminated string starting at] {"total_pages":1,"current_page":1,"total_records":312,"records":[{"id":"5ff85bb0fe131d001c666a7a","field_1596":"p100k051","field_1596_raw":"p100k051","field_1571":"51","field_1571_raw":51,"field_1572":"06/01/2022","field_1572_raw":{"date":"01/06/2022","date_formatted":"06/01/2022","hours":"12","minutes":"00","am_pm":"AM","unix_timestamp":1641427200000,"iso_timestamp":"2022-01-06T00:00:00.000Z","timestamp":"01/06/2022 12:00 am","time":720},"field_1573":"08/01-14/01","field_1573_raw":"08/01-14/01","field_1594":"45,4","field_1594_raw":45.3890107647569,"field_1595":"223","field_1595_raw":222.8558220664271,"field_1597":"Positief getest per 100.000","field_1597_raw":"Positief getest per 100.000","field_1574":"44","field_1574_raw":44.008728966499916,"field_1575":"222","field_1575_raw":221.92475160333692,"field_1576":"289","field_1576_raw":289.00867202456044,"field_1577":"226","field_1577_raw":225.92544009707763,"field_1578":"242","field